### How to setup the spark context.

Download the latest spark https://www.apache.org/dyn/closer.lua/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz

Go Inside and run the spark-shell command. This will download all the relevant jars.

A SparkContext is a client of Spark’s execution environment and it acts as the master of the Spark application. SparkContext sets up internal services and establishes a connection to a Spark execution environment.

You may want to avoid hard-coding certain configurations in a SparkConf. For instance, if you’d like to run the same application with different masters or different amounts of memory. Spark allows you to simply create an empty conf:

    val sc = new SparkContext(new SparkConf())

Then, you can supply configuration values at runtime:

    ./bin/spark-submit --name "My app" --master local[4] --conf spark.eventLog.enabled=false --conf "spark.executor.extraJavaOptions=-XX:+PrintGCDetails -XX:+PrintGCTimeStamps" myApp.jar

### A quick word on spark tools: sbt, and spark-submit, spark-shell, pyspark

Sbt is the built tool for building scala applications.

You will need to submit applications to your spark cluster using the spark-submit.

Spark-shell will help you in understanding the code execution flow. It is similar to ipython for spark.

FInally the bigger question of whether to use scala. According to me the question is do you already have a lot of legacy code in python, and how comfortable is your team to go into the typed environment of scala. Do you believe that strict typing is your friend, because that will be an extra cognitive load. If you ask me, making the upfront investment in typing will help you in your data debugging process. I highly recommend this is the data that you are getting is ambiguous and is likely to change over time.

### Difference between a transformation and action

In pandas everything is a transformation.

Transformations are executed on demand.(Lazy computation)
Ex: filter(), union()

An Action will return a non-RDD type (your stored value types usually)
Actions triggers execution using lineage graph to load the data into original RDD
Ex: count(), first()

### Difference: creating a pandas DF and a Spark DF.

DataFrames generally refer to a data structure, which is tabular in nature. It represents rows, each of which consists of a number of observations. Rows can have a variety of data formats (heterogeneous), whereas a column can have data of the same data type (homogeneous). DataFrames usually contain some metadata in addition to data; for example, column and row names.

#### Pandas:

![title](img/pandas_read_csv.png)

#### Spark

In [1]:
val house_prices_df = spark.read
    .format("csv")                                    // this is a csv file.
    .option("header", "true")                         // the file contains headers
    .option("inferSchema", true)                      // read the schema
    .load("/home/jovyan/data/house-prices/train.csv") // now load the file.

val melb_data = spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", true)
    .load("/home/jovyan/data/melbourne_housing_snapshot/melb_data.csv")

house_prices_df = [Id: int, MSSubClass: int ... 79 more fields]
melb_data = [Suburb: string, Address: string ... 19 more fields]


[Suburb: string, Address: string ... 19 more fields]

Taking a look at the dataframe.

In [2]:
house_prices_df.show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [3]:
house_prices_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

### Features of Spark Dataframes

    * DataFrames are distributed in nature, which makes it a fault tolerant and highly available data structure.
    * Lazy evaluation is an evaluation strategy which holds the evaluation of an expression until its value is needed. It avoids repeated evaluation. Lazy evaluation in Spark means that the execution will not start until an action is triggered. In Spark, the picture of lazy evaluation comes when Spark transformations occur.
    * DataFrames are immutable in nature. By immutable, I mean that it is an object whose state cannot be modified after it is created. But we can transform its values by applying a certain transformation, like in RDDs.


### Describing a particular column

#### Pandas

![Describing in pandas](./img/pandas_describe.png)

#### Spark

In [4]:
house_prices_df.describe("MSSubClass").show()

+-------+------------------+
|summary|        MSSubClass|
+-------+------------------+
|  count|              1460|
|   mean|56.897260273972606|
| stddev| 42.30057099381045|
|    min|                20|
|    max|               190|
+-------+------------------+



### Get the dataframe shape and columns

#### Pandas

![](./img/pandas_shape_columns.png)

#### Spark

Getting the number of samples is an action and hence be mindful

In [5]:
house_prices_df.count

1460

Getting the number of features is a no big deal

In [6]:
house_prices_df.columns.size

81

In [7]:
house_prices_df.columns

Array(Id, MSSubClass, MSZoning, LotFrontage, LotArea, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, OverallQual, OverallCond, YearBuilt, YearRemodAdd, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, MasVnrArea, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinSF1, BsmtFinType2, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, Heating, HeatingQC, CentralAir, Electrical, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, KitchenQual, TotRmsAbvGrd, Functional, Fireplaces, FireplaceQu, GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, PavedDr...

### Changing the column names of the dataframes

#### Pandas

![](./img/change_column_names.png)

#### Spark

In [8]:
val df = Seq((1L, "a", "foo", 3.0)).toDF
df.printSchema

root
 |-- _1: long (nullable = false)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: double (nullable = false)



df = [_1: bigint, _2: string ... 2 more fields]


[_1: bigint, _2: string ... 2 more fields]

In [9]:
val newNames = Seq("id", "x1", "x2", "x3")
val dfRenamed = df.toDF(newNames: _*)

dfRenamed.show

+---+---+---+---+
| id| x1| x2| x3|
+---+---+---+---+
|  1|  a|foo|3.0|
+---+---+---+---+



newNames = List(id, x1, x2, x3)
dfRenamed = [id: bigint, x1: string ... 2 more fields]


[id: bigint, x1: string ... 2 more fields]

In [10]:
import org.apache.spark.sql.functions.col

val lookup = Map("Id" -> "id", "SalePrice" -> "SalePriceDollars")

val changed_cols_df = house_prices_df.select(
    house_prices_df.columns
    .map(
        c => col(c).as(lookup.getOrElse(c, c))
    ): _*)

lookup = Map(Id -> id, SalePrice -> SalePriceDollars)
changed_cols_df = [id: int, MSSubClass: int ... 79 more fields]


[id: int, MSSubClass: int ... 79 more fields]

In [11]:
changed_cols_df.printSchema

root
 |-- id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

#### Unique Values

#### Pandas

![](./img/unique_values.png)

#### Spark

In [12]:
house_prices_df.select("MSSubClass").distinct().show()

+----------+
|MSSubClass|
+----------+
|        85|
|       190|
|        20|
|        40|
|       120|
|        50|
|        45|
|        80|
|       160|
|        70|
|        60|
|        90|
|        75|
|        30|
|       180|
+----------+



For large data when you are only interested in the count of unique values.

In [54]:
import org.apache.spark.sql.functions.countDistinct

house_prices_df.select(countDistinct('MSSubClass)).show(3)

+--------------------------+
|count(DISTINCT MSSubClass)|
+--------------------------+
|                        15|
+--------------------------+



You can also do count approximate

#### Value Counts

ref: https://stackoverflow.com/a/37949565/5417164

#### Pandas

![](./img/value_counts.png)

#### Spark

In [56]:
import org.apache.spark.sql.functions.count

house_prices_df
    .groupBy("MSSubClass")  // groupby your class
    .count()                // count the values, this should create a dedicated count column
    .orderBy($"count" desc)       // orderby the count column
    .show()

+----------+-----+
|MSSubClass|count|
+----------+-----+
|        20|  536|
|        60|  299|
|        50|  144|
|       120|   87|
|        30|   69|
|       160|   63|
|        70|   60|
|        80|   58|
|        90|   52|
|       190|   30|
|        85|   20|
|        75|   16|
|        45|   12|
|       180|   10|
|        40|    4|
+----------+-----+



### Order by and group by

#### Pandas

![](./img/orderbuy_grpby.png)

#### Spark

In [14]:
import org.apache.spark.sql.functions.count

house_prices_df
    .groupBy($"MSSubClass")                   // Count number of occurrences of each word
    .agg(count("*") as "numOccurances")       // SQL: SELECT COUNT(DISTINCT MSSubClass) AS numOccurances FROM house_prices_df
    .orderBy($"numOccurances" desc).show()

+----------+-------------+
|MSSubClass|numOccurances|
+----------+-------------+
|        20|          536|
|        60|          299|
|        50|          144|
|       120|           87|
|        30|           69|
|       160|           63|
|        70|           60|
|        80|           58|
|        90|           52|
|       190|           30|
|        85|           20|
|        75|           16|
|        45|           12|
|       180|           10|
|        40|            4|
+----------+-------------+



### Filtering data

#### Pandas

![](./img/filtering.png)

#### Spark

In [15]:
val highSubClass = house_prices_df.filter($"MSSubClass" > 100)

highSubClass = [Id: int, MSSubClass: int ... 79 more fields]


[Id: int, MSSubClass: int ... 79 more fields]

In [16]:
highSubClass.show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

### Membership in dataframe

#### Pandas

![](./img/membership.png)

#### Spark

In [17]:
val presentList = List("20","60") 
val nopresentList = List("20000") 
val validMembership = house_prices_df.filter($"MSSubClass".isin(presentList:_*))
val invalidMembership = house_prices_df.filter($"MSSubClass".isin(nopresentList:_*))

presentList = List(20, 60)
nopresentList = List(20000)
validMembership = [Id: int, MSSubClass: int ... 79 more fields]
invalidMembership = [Id: int, MSSubClass: int ... 79 more fields]


[Id: int, MSSubClass: int ... 79 more fields]

You can see that the count below is the sum of 536 + 299

In [18]:
validMembership.count

835

In [19]:
invalidMembership.count

0

### Missing value imputation

refs: https://stackoverflow.com/a/40059453/5417164 

https://medium.com/@mrpowers/dealing-with-null-in-spark-cfdbb12f231e

In [20]:
melb_data.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: double (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: double (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: double (nullable = true)



In [21]:
melb_data.show(3)

+----------+---------------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|        Address|Rooms|Type|    Price|Method|SellerG|     Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+---------------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|   85 Turner St|    2|   h|1480000.0|     S| Biggin|3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford|25 Bloomburg St|    2|   h|1035000.0|     S| Biggin|4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0|   156.0|        79.0|   1

#### Pandas

![](./img/fillna.png)

Similar to pandas you can replace the na values

In [22]:
val imputed_melb_data = melb_data.na.fill(1964.0, Seq("YearBuilt"))

imputed_melb_data = [Suburb: string, Address: string ... 19 more fields]


[Suburb: string, Address: string ... 19 more fields]

In [23]:
imputed_melb_data.show(5)

+----------+----------------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|         Address|Rooms|Type|    Price|Method|SellerG|     Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+----------------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|    85 Turner St|    2|   h|1480000.0|     S| Biggin|3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|   1964.0|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford| 25 Bloomburg St|    2|   h|1035000.0|     S| Biggin|4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0|   156.0|        79.0

Using an imputer

#### Pandas

![](./img/imputer.png)

#### Spark

In [24]:
import org.apache.spark.ml.feature.Imputer

val features_in_focus = Array("Rooms", "Bathroom", "Landsize", "BuildingArea",
                              "YearBuilt", "Lattitude", "Longtitude")

val imputer = new Imputer()
  .setInputCols(features_in_focus)
  .setOutputCols(features_in_focus.map(c => s"${c}_imputed"))
  .setStrategy("mean")

imputer.fit(melb_data).transform(melb_data)

Name: java.lang.IllegalArgumentException
Message: requirement failed: Column Rooms must be of type equal to one of the following types: [DoubleType, FloatType] but was actually of type IntegerType.
StackTrace:   at scala.Predef$.require(Predef.scala:224)
  at org.apache.spark.ml.util.SchemaUtils$.checkColumnTypes(SchemaUtils.scala:58)
  at org.apache.spark.ml.feature.ImputerParams$$anonfun$2.apply(Imputer.scala:76)
  at org.apache.spark.ml.feature.ImputerParams$$anonfun$2.apply(Imputer.scala:74)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
  at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
  at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
  at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)
  at org.apache.spark.ml

As you can see above the features must be for double type or floattype. But the Rooms feature is of type Integer and hence we will need to convert that.

In [25]:
import org.apache.spark.sql.types.DoubleType

val melb_data2 = melb_data
    .withColumn("_Rooms", melb_data("Rooms").cast(DoubleType))
    .drop("Rooms")
    .withColumnRenamed("_Rooms", "Rooms")

melb_data2 = [Suburb: string, Address: string ... 19 more fields]


lastException: Throwable = null


[Suburb: string, Address: string ... 19 more fields]

In [26]:
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql.functions.col

val features_in_focus = Array("Rooms", "Bathroom", "Landsize", "BuildingArea",
                              "YearBuilt", "Lattitude", "Longtitude")
val features_in_focus_imputed = features_in_focus.map(c => s"${c}_imputed")

val imputer = new Imputer()
  .setInputCols(features_in_focus)
  .setOutputCols(features_in_focus_imputed)
  .setStrategy("mean")

val imputed_melb_data = imputer.fit(melb_data2).transform(melb_data2)

imputed_melb_data.select(features_in_focus_imputed.map(name => col(name)):_*).show(5)

+-------------+----------------+----------------+--------------------+------------------+-----------------+------------------+
|Rooms_imputed|Bathroom_imputed|Landsize_imputed|BuildingArea_imputed| YearBuilt_imputed|Lattitude_imputed|Longtitude_imputed|
+-------------+----------------+----------------+--------------------+------------------+-----------------+------------------+
|          2.0|             1.0|           202.0|  151.96764988779805|1964.6842169408897|         -37.7996|          144.9984|
|          2.0|             1.0|           156.0|                79.0|            1900.0|         -37.8079|          144.9934|
|          3.0|             2.0|           134.0|               150.0|            1900.0|         -37.8093|          144.9944|
|          3.0|             2.0|            94.0|  151.96764988779805|1964.6842169408897|         -37.7969|          144.9969|
|          4.0|             1.0|           120.0|               142.0|            2014.0|         -37.8072|    

features_in_focus = Array(Rooms, Bathroom, Landsize, BuildingArea, YearBuilt, Lattitude, Longtitude)
features_in_focus_imputed = Array(Rooms_imputed, Bathroom_imputed, Landsize_imputed, BuildingArea_imputed, YearBuilt_imputed, Lattitude_imputed, Longtitude_imputed)
imputer = imputer_fd2f4820db4f
imputed_melb_data = [Suburb: string, Address: string ... 26 more fields]


[Suburb: string, Address: string ... 26 more fields]

### Discretization and Binning

#### Pandas

![](./img/binning.png)

#### Spark

In [27]:
import org.apache.spark.ml.feature.Bucketizer

val splits = Array(Double.NegativeInfinity, -0.5, 0.0, 0.5, Double.PositiveInfinity)

val data = Array(-999.9, -0.5, -0.3, 0.0, 0.2, 999.9)
val dataFrame = spark.createDataFrame(data.map(Tuple1.apply)).toDF("features")

println("showing the dataframe")
dataFrame.show()

val bucketizer = new Bucketizer()
  .setInputCol("features")
  .setOutputCol("bucketedFeatures")
  .setSplits(splits)

// Transform original data into its bucket index.
val bucketedData = bucketizer.transform(dataFrame)

println(s"Bucketizer output with ${bucketizer.getSplits.length-1} buckets")
bucketedData.show()

showing the dataframe
+--------+
|features|
+--------+
|  -999.9|
|    -0.5|
|    -0.3|
|     0.0|
|     0.2|
|   999.9|
+--------+

Bucketizer output with 4 buckets
+--------+----------------+
|features|bucketedFeatures|
+--------+----------------+
|  -999.9|             0.0|
|    -0.5|             1.0|
|    -0.3|             1.0|
|     0.0|             2.0|
|     0.2|             2.0|
|   999.9|             3.0|
+--------+----------------+



splits = Array(-Infinity, -0.5, 0.0, 0.5, Infinity)
data = Array(-999.9, -0.5, -0.3, 0.0, 0.2, 999.9)
dataFrame = [features: double]
bucketizer = bucketizer_afd411a4ab1f
bucketedData = [features: double, bucketedFeatures: double]


[features: double, bucketedFeatures: double]

In [28]:
val data = Array(20.0, 22.0, 25.0, 27.0, 21.0, 23.0, 37.0, 31.0, 61.0, 45.0, 41.0, 32.0)
val bins = Array(18.0, 25.0, 35.0, 60.0, 100.0)

// val dataFrame = spark.createDataFrame(data.map(Tuple1.apply)).toDF("features")
val dataFrame = spark.createDataFrame(data.map(Tuple1.apply)).toDF("features")

println("showing the dataframe")
dataFrame.show()

val bucketizer = new Bucketizer()
  .setInputCol("features")
  .setOutputCol("bucketedFeatures")
  .setSplits(bins)

// Transform original data into its bucket index.
val bucketedData = bucketizer.transform(dataFrame)

println(s"Bucketizer output with ${bucketizer.getSplits.length-1} buckets")
bucketedData.show()

showing the dataframe
+--------+
|features|
+--------+
|    20.0|
|    22.0|
|    25.0|
|    27.0|
|    21.0|
|    23.0|
|    37.0|
|    31.0|
|    61.0|
|    45.0|
|    41.0|
|    32.0|
+--------+

Bucketizer output with 4 buckets
+--------+----------------+
|features|bucketedFeatures|
+--------+----------------+
|    20.0|             0.0|
|    22.0|             0.0|
|    25.0|             1.0|
|    27.0|             1.0|
|    21.0|             0.0|
|    23.0|             0.0|
|    37.0|             2.0|
|    31.0|             1.0|
|    61.0|             3.0|
|    45.0|             2.0|
|    41.0|             2.0|
|    32.0|             1.0|
+--------+----------------+



data = Array(20.0, 22.0, 25.0, 27.0, 21.0, 23.0, 37.0, 31.0, 61.0, 45.0, 41.0, 32.0)
bins = Array(18.0, 25.0, 35.0, 60.0, 100.0)
dataFrame = [features: double]
bucketizer = bucketizer_a14e358ca67c
bucketedData = [features: double, bucketedFeatures: double]


[features: double, bucketedFeatures: double]

### Getting a particular data

ref: https://stackoverflow.com/a/35720457/5417164

#### Pandas

![](./img/iloc.png)

#### Spark

In [61]:
val result = house_prices_df.
    filter(line => line(0) == 1)
    .select("MSSubClass").collect()

result = Array([60])


Array([60])

In [31]:
result

Array([60])

### Sorting

ref: https://stackoverflow.com/a/32052881/5417164

#### Pandas

![](./img/sorting.png)

#### Spark

In [32]:
import org.apache.spark.sql.functions._
val sortedbyMsSubclass = house_prices_df.sort(desc("MSSubClass"))

sortedbyMsSubclass = [Id: int, MSSubClass: int ... 79 more fields]


[Id: int, MSSubClass: int ... 79 more fields]

In [33]:
val columnNames = Seq("MSSubClass", "SalePrice")
sortedbyMsSubclass.select(columnNames.map(c => col(c)): _*).show(3)

+----------+---------+
|MSSubClass|SalePrice|
+----------+---------+
|       190|   133900|
|       190|   137000|
|       190|    84500|
+----------+---------+
only showing top 3 rows



columnNames = List(MSSubClass, SalePrice)


List(MSSubClass, SalePrice)

### Reshaping and Pivoting

#### Pandas

![](./img/pivot.png)

#### Spark

In [34]:
// create RDD for products
val data = sc.parallelize(Seq(
    ("memories","book","q1",10),
    ("dreams","book","q2",20),
    ("reflections","book","q3",30),
    ("how to build a house","book","q4",40),
    ("wonderful life","music","q1",10),
    ("million miles","music","q2",20),
    ("run away","music","q3",30),
    ("mind and body","music","q4",40)
))

// convert the RDD to DataFrame
val df_products = spark.createDataFrame(data).toDF("product","category","quarter","profit")
df_products.show()

// index column : category
// value column : profit
// pivot column : quarter
// agg function : sum

// apply pivot on DataFrame DataFrame
df_products.groupBy("category").pivot("quarter").sum("profit").show()

+--------------------+--------+-------+------+
|             product|category|quarter|profit|
+--------------------+--------+-------+------+
|            memories|    book|     q1|    10|
|              dreams|    book|     q2|    20|
|         reflections|    book|     q3|    30|
|how to build a house|    book|     q4|    40|
|      wonderful life|   music|     q1|    10|
|       million miles|   music|     q2|    20|
|            run away|   music|     q3|    30|
|       mind and body|   music|     q4|    40|
+--------------------+--------+-------+------+

+--------+---+---+---+---+
|category| q1| q2| q3| q4|
+--------+---+---+---+---+
|   music| 10| 20| 30| 40|
|    book| 10| 20| 30| 40|
+--------+---+---+---+---+



data = ParallelCollectionRDD[105] at parallelize at <console>:43
df_products = [product: string, category: string ... 2 more fields]


[product: string, category: string ... 2 more fields]

### Merges and Joins

#### Pandas

![](./img/merges.png)

#### Spark

In [35]:
val llist = Seq(("bob", "2015-01-13", 4), ("alice", "2015-04-23",10))
val left = llist.toDF("name","date","duration")
val right = Seq(("alice", 100),("bob", 23)).toDF("name","upload")

val df = left.join(right, Seq("name"))
df.show()

+-----+----------+--------+------+
| name|      date|duration|upload|
+-----+----------+--------+------+
|  bob|2015-01-13|       4|    23|
|alice|2015-04-23|      10|   100|
+-----+----------+--------+------+



llist = List((bob,2015-01-13,4), (alice,2015-04-23,10))
left = [name: string, date: string ... 1 more field]
right = [name: string, upload: int]
df = [name: string, date: string ... 2 more fields]


[name: string, date: string ... 2 more fields]

#### Concatenating and appending to the dataframe.

#### Pandas

![](./img/append.png)

#### Spark

In [36]:
val llist = Seq(("bob", "2015-01-13", 4), ("alice", "2015-04-23",10))
val arr = llist.toDF("name","date","duration")
val appended = arr.union(arr.toDF())
appended.show()

+-----+----------+--------+
| name|      date|duration|
+-----+----------+--------+
|  bob|2015-01-13|       4|
|alice|2015-04-23|      10|
|  bob|2015-01-13|       4|
|alice|2015-04-23|      10|
+-----+----------+--------+



llist = List((bob,2015-01-13,4), (alice,2015-04-23,10))
arr = [name: string, date: string ... 1 more field]
appended = [name: string, date: string ... 1 more field]


[name: string, date: string ... 1 more field]

### Function application, transformations and mapping

By using user defined functions

In [37]:
import org.apache.spark.sql.functions.udf

In [38]:
def morePrecision(price: Integer): Float = price.toFloat

// we use the method name followed by a "_" to indicate we want a reference
// to the method, not call it
val morePrecisionUdf = udf(morePrecision _)

val converted_df = house_prices_df.select(morePrecisionUdf(house_prices_df("SalePrice")))

morePrecisionUdf = UserDefinedFunction(<function1>,FloatType,Some(List(IntegerType)))
converted_df = [UDF(SalePrice): float]


morePrecision: (price: Integer)Float


[UDF(SalePrice): float]

In [39]:
converted_df.show(3)

+--------------+
|UDF(SalePrice)|
+--------------+
|      208500.0|
|      181500.0|
|      223500.0|
+--------------+
only showing top 3 rows



Applying the udf on the same df. ie. creating a new feature by transforming another column.

In [40]:
house_prices_df.withColumn("MorePrecisionSalePrice", morePrecisionUdf('SalePrice)).show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+----------------------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborho

Applying some transformation on all the data.

In [41]:
import org.apache.spark.sql.functions.{col, upper}

val df = sc.parallelize(
  Seq(("a", "B", "c"), ("D", "e", "F"))).toDF("x", "y", "z")
df.select(df.columns.map(c => upper(col(c)).alias(c)): _*).show

+---+---+---+
|  x|  y|  z|
+---+---+---+
|  A|  B|  C|
|  D|  E|  F|
+---+---+---+



df = [x: string, y: string ... 1 more field]


[x: string, y: string ... 1 more field]

### A look at datasets

The Datasets API provides the benefits of RDDs (strong typing, ability to use powerful lambda functions) with the benefits of Spark SQL’s optimized execution engine. You can define a Dataset JVM objects and then manipulate them using functional transformations (map, flatMap, filter, and so on) similar to an RDD. The benefits is that, unlike RDDs, these transformations are now applied on a structured and strongly typed distributed collection that allows Spark to leverage Spark SQL’s execution engine for optimization.

In [42]:
import org.apache.spark.sql.functions._

val wordsDataset = sc.parallelize(Seq("Spark I am your father", "May the spark be with you", "Spark I am your father")).toDS()
val result = wordsDataset
              .flatMap(_.split(" "))               // Split on whitespace
              .filter(_ != "")                     // Filter empty words
              .map(_.toLowerCase())
              .toDF()                              // Convert to DataFrame to perform aggregation / sorting
              .groupBy($"value")                   // Count number of occurrences of each word
              .agg(count("*") as "numOccurances")
              .orderBy($"numOccurances" desc)      // Show most common words first
result.show()

+------+-------------+
| value|numOccurances|
+------+-------------+
| spark|            3|
|father|            2|
|  your|            2|
|    am|            2|
|     i|            2|
|  with|            1|
|   may|            1|
|   you|            1|
|    be|            1|
|   the|            1|
+------+-------------+



wordsDataset = [value: string]
result = [value: string, numOccurances: bigint]


[value: string, numOccurances: bigint]

### Additional Capabilities: Streaming.

In [43]:

import org.apache.spark.sql.types.{StringType, StructType, StructField, IntegerType}

val userSchema = new StructType().add("Suburb", "string").add("Address", "string")
val csvDF = spark
  .readStream
  .option("sep", ",")
  .schema(userSchema)      // Specify schema of the csv files
  .option("maxFilesPerTrigger", 1)
  .csv("/home/jovyan/data/melbourne_housing_snapshot/")

userSchema = StructType(StructField(Suburb,StringType,true), StructField(Address,StringType,true))
csvDF = [Suburb: string, Address: string]


[Suburb: string, Address: string]

In [44]:
// Same query as staticInputDF
import org.apache.spark.sql.functions._

val streamingCountsDF = 
  csvDF
    .groupBy($"Suburb", window($"Address", "1 minute"))
    .count()

streamingCountsDF.isStreaming

streamingCountsDF = [Suburb: string, window: struct<start: timestamp, end: timestamp> ... 1 more field]


true

In [45]:
spark.conf.set("spark.sql.shuffle.partitions", "1")  // keep the size of shuffles small

val query =
  streamingCountsDF
    .writeStream
    .format("memory")        // memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     // counts = name of the in-memory table
    .outputMode("complete")  // complete = all the counts should be in the table
    .start()

query = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1060be6


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@1060be6

In [46]:
spark.sql("select * from counts").show()

+------+------+-----+
|Suburb|window|count|
+------+------+-----+
+------+------+-----+



### A sample machine learning dataset.

In [47]:
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.evaluation.RegressionEvaluator

val bike_sharing_df = spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", true)
    .load("/home/jovyan/data/bike-sharing/hour.csv")

bike_sharing_df = [instant: int, dteday: timestamp ... 15 more fields]


[instant: int, dteday: timestamp ... 15 more fields]

In [48]:
val featureCols = Array("season", "yr", "mnth", "hr", 
                        "holiday", "weekday", "workingday",
                        "weathersit", "temp", "atemp",
                        "hum", "windspeed", "cnt")
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol("features")
val dataDF = assembler.transform(bike_sharing_df)
val dataDF1 = dataDF.withColumnRenamed("cnt", "label")

val Array(train, test) = dataDF1.randomSplit(Array(0.8, 0.2))

val lr = new LinearRegression()
    .setMaxIter(1000)
    .setRegParam(0.3)
    .setElasticNetParam(0.8)

//Fit the model
val lrModel = lr.fit(train)

featureCols = Array(season, yr, mnth, hr, holiday, weekday, workingday, weathersit, temp, atemp, hum, windspeed, cnt)
assembler = vecAssembler_e03f82143a97
dataDF = [instant: int, dteday: timestamp ... 16 more fields]
dataDF1 = [instant: int, dteday: timestamp ... 16 more fields]
train = [instant: int, dteday: timestamp ... 16 more fields]
test = [instant: int, dteday: timestamp ... 16 more fields]
lr = linReg_4e3773869c3c
lrModel = linReg_4e3773869c3c


linReg_4e3773869c3c